In [44]:
import os
import warnings
from tqdm import tqdm

import pandas as pd
import numpy as np
import pyarrow.feather as feather

from sklearn.model_selection import (
    KFold, StratifiedKFold, GridSearchCV, 
    RandomizedSearchCV, RepeatedStratifiedKFold)

from sklearn.metrics import mean_squared_error
from sklearn.ensemble import (
    BaggingRegressor, RandomForestRegressor, 
    AdaBoostRegressor, GradientBoostingRegressor)

warnings.filterwarnings('ignore')

In [45]:
data_list = os.listdir("./data")
data_list

['FIFA_train.csv',
 'submission.csv',
 'submission_baseline_rf.csv',
 'FIFA_test.csv',
 'data.ftr']

In [46]:
train = pd.read_feather('./data/' + data_list[4])
test = pd.read_csv('./data/' + data_list[3])

In [47]:
# 데이터 분리

x_train = train.drop('value', axis = 1)
y_train = train['value']

KeyError: "['value'] not found in axis"

In [48]:
# kfold 
kfold = KFold(n_splits = 7, shuffle = True, random_state = 120)

In [49]:
# 회귀에서는 쓰지 않는다. 분류에서만 사용

stratifiedkfolf = StratifiedKFold()

In [50]:
# kflod 에서의 데이터 분리 (어떤 데이터를 분리할 것인가) train, validataion 을 반환
# could not convert string to float: 'De Gea' 오류는 데이터에 숫자형태 외에 데이터가 있어서 발생

for i, (t, v) in enumerate (kfold.split(train)) : 
    
    list = []
    trn = train.iloc[t]
    val = train.iloc[v]
    
    # x, y 분리
    x_tr = trn.drop('values', axis = 1)
    y_tr = trn['value']
    
    x_val = val.drop('value', axis = 1)
    y_val = val['value']
    
    # 모델 학습
    rf = RandomForestRegressor(n_estimators = 300, random_state = 130)
    rf.fit(x_tr, y_tr)
    
    # 예측
    pred = rf.predict(x_val)
    pred = np.expm1(pred)
    
    y_val = np.expm1(y_val)
    
    # rmse
    mse = mean_squared_error(y_val, pred)
    rmse = np.sqrt(mse)
    
    list.append(rmse)
    
    print(f'{i + 1}번 모델 rmse : {rmse}')

KeyError: "['values'] not found in axis"

In [51]:
# GridSearch

rf = RandomForestRegressor(random_state = 120)

In [30]:
params = {'n_estimators' : [300, 400, 500], 
         'min_samples_split' : [2, 3, 4],
         'min_samples_leaf' : [1, 2, 3]}

In [34]:
cv = KFold(n_splits = 3, shuffle = True, random_state = 120)

In [39]:
grid = GridSearchCV(rf, 
                    param_grid = params, 
                    cv = cv, 
                    scoring = 'neg_mean_squared_error', 
                    
                    n_jobs = -1)

In [41]:
# 연상량이 많아 큰 데이터에서는 쓰기 어려워 딥러닝에서는 잘 안쓰인다.
# 따라서 머신러닝에서만 사용한다.

grid.fit(x_train, y_train)

NameError: name 'x_train' is not defined

In [42]:
print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)
print(grid.best_index_)

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

In [43]:
RandomForestRegressor(**grid.best_params_)

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'